#Leitura dos dados

In [1]:
!pip install pyspark==3.3.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 35 kB/s 
     |████████████████████████████████| 199 kB 55.4 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=b5cacaa4354e10b5e21772018ee58c049eedf734026cdd253af4c9857def6649
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.master('local[*]').appName("Transformacao com Spark").getOrCreate()

spark

In [4]:
!wget "https://caelum-online-public.s3.amazonaws.com/challenge-spark/semana-2.zip" && unzip semana-2.zip -d dados/

--2022-12-01 16:24:17--  https://caelum-online-public.s3.amazonaws.com/challenge-spark/semana-2.zip
Resolving caelum-online-public.s3.amazonaws.com (caelum-online-public.s3.amazonaws.com)... 3.5.0.128, 52.216.160.227, 54.231.133.169, ...
Connecting to caelum-online-public.s3.amazonaws.com (caelum-online-public.s3.amazonaws.com)|3.5.0.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2763652 (2.6M) [application/zip]
Saving to: ‘semana-2.zip’

semana-2.zip        100%[===================>]   2.63M  6.12MB/s    in 0.4s    

2022-12-01 16:24:18 (6.12 MB/s) - ‘semana-2.zip’ saved [2763652/2763652]

Archive:  semana-2.zip
   creating: dados/dataset_transformado_parquet/
  inflating: dados/dataset_transformado_parquet/_SUCCESS  
  inflating: dados/dataset_transformado_parquet/._SUCCESS.crc  
  inflating: dados/dataset_transformado_parquet/part-00000-00341ba7-0a7c-4fef-a81e-1066725a64b1-c000.snappy.parquet  
  inflating: dados/dataset_transformado_parquet/.part-0000

In [5]:
dados = spark.read.parquet('/content/dados/dataset_transformado_parquet')

In [6]:
dados.show(truncate=False)

+------------------------------------+-----+----------+---------+---------+-----------------------------------------------------------------------------------------------------------------------------------------+-------+------+------------+------------+-----------+----+-----------------------+------------+----------+----+-----+------+
|id                                  |andar|area_total|area_util|banheiros|caracteristicas                                                                                                                          |quartos|suites|tipo_anuncio|tipo_unidade|tipo_uso   |vaga|bairro                 |zona        |condominio|iptu|tipo |valor |
+------------------------------------+-----+----------+---------+---------+-----------------------------------------------------------------------------------------------------------------------------------------+-------+------+------------+------------+-----------+----+-----------------------+------------+----------+----+

In [11]:
dados.printSchema()

root
 |-- id: string (nullable = true)
 |-- andar: long (nullable = true)
 |-- area_total: string (nullable = true)
 |-- area_util: string (nullable = true)
 |-- banheiros: long (nullable = true)
 |-- caracteristicas: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- quartos: long (nullable = true)
 |-- suites: long (nullable = true)
 |-- tipo_anuncio: string (nullable = true)
 |-- tipo_unidade: string (nullable = true)
 |-- tipo_uso: string (nullable = true)
 |-- vaga: long (nullable = true)
 |-- bairro: string (nullable = true)
 |-- zona: string (nullable = true)
 |-- condominio: string (nullable = true)
 |-- iptu: string (nullable = true)
 |-- tipo: string (nullable = true)
 |-- valor: string (nullable = true)



#Analisando a diferença entre a quantidade da dados não nulos armazenados nas colunas 'area_util' e 'area_total'

In [7]:
from pyspark.sql import functions as f

In [24]:
dados.select('area_total').where(f.col('area_total').isNotNull()).count()

57368

In [25]:
dados.select('area_util').where(f.col('area_util').isNotNull()).count()

66562

In [28]:
dados = dados.drop('area_total')

In [29]:
dados.printSchema()

root
 |-- id: string (nullable = true)
 |-- andar: long (nullable = true)
 |-- area_util: string (nullable = true)
 |-- banheiros: long (nullable = true)
 |-- caracteristicas: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- quartos: long (nullable = true)
 |-- suites: long (nullable = true)
 |-- tipo_anuncio: string (nullable = true)
 |-- tipo_unidade: string (nullable = true)
 |-- tipo_uso: string (nullable = true)
 |-- vaga: long (nullable = true)
 |-- bairro: string (nullable = true)
 |-- zona: string (nullable = true)
 |-- condominio: string (nullable = true)
 |-- iptu: string (nullable = true)
 |-- tipo: string (nullable = true)
 |-- valor: string (nullable = true)

